# Introduction à la fouille de texte en Python

---

Le dépôt Github durable de ce cours, ainsi que les textes et le code source, sont disponibles [à l'adresse suivante](https://github.com/paulhectork/cours_ens2024_fouille_de_texte/). Il contient aussi des références bibliographiques, et des liens vers d'autres tutoriels pour aller plus loin.

---

# Introduction

Dans ce cours, nous allons faire une analyse statistique de trois romans de Virginia Woolf: *Mrs. Dalloway* (1925), *To the lighthouse* (1927), *The Waves* (1931). Il s'agit de voir si le style de Woolf évolue, et si oui, comment. On analyse les textes par "lecture distante" (par opposition à une lecture "proche", humaine): on ne *lit pas* les romans, mais on les prend comme corpus de données qui peuvent être approchées par des méthodes computationnelles. On peut donc travailler sur de grands corpus, ici de trois romans entiers:

- **Mrs. Dalloway** décrit la journée de Clarissa Dalloway, une femme britannique de classe sociale supérieure, avant la 1e Guerre Mondiale. Le roman suit le flux de conscience de Clarissa, et il est souvent considéré comme une réponse à *Ulysses* de Joyce, qui suit lui aussi un personnage pendant une journée (livre qu'elle critique abondamment par ailleurs).
- **To the lighthouse** décrit quelques journées de la famille Ramsay et de leurs invité.e.s dans leur maison de villégiature, sur l'île de Syke en Écosse. Le roman s'étale sur une période de 10 ans, avant et après la Première Guerre Mondiale. Il entrecroise différentes histoires familiales, contient peu de dialogues directs et se concentre sur les pensées des personnages.
- **The Waves** est composé de solliloques et dialogues de six narrateur.ice.s qui s'étalent sur plusieurs années, de l'enfance à l'âge adulte. C'est généralement considéré comme son roman le plus expérimental, et le plus difficile à lire.

Notre question de recherche:

> Le style de Woolf évolue-t-il d'un roman à l'autre?
> Est-ce que le caractère "expérimental" et "difficile" de *The Waves* ressort d'une analyse statistique ?

Le choix du corpus, et une partie de la méthodologie, sont inspirés de:

> Hussein, K. & Kadhim, R. (2020). A Corpus-Based Stylistic Identification of Lexical Density Profile of Three Novels by Virginia Woolf: The Waves, Mrs. Dalloway and To the Lighthouse. *International Journal of Psychosocial Rehabilitation*. 24. pp. 6688-9702. En accès libre à [cette addresse](https://www.researchgate.net/publication/343797320_A_Corpus-Based_Stylistic_Identification_of_Lexical_Density_Profile_of_Three_Novels_by_Virginia_Woolf_The_Waves_Mrs_Dalloway_and_To_the_Lighthouse)

---

# Le programme des festivités

## Pipeline

Ces trois romans ont été téléchargés depuis Archive.org en texte brut (c'est-à-dire, sans éléments de mise en page) et légèrement nettoyés (les en-têtes, fin de page, la pagination et la séparation de chapitres sont supprimés). La chaîne de traitement est la suivante:

- **on ouvre les fichiers** et on lit leur contenu
- **on les simplifie** rapidement
- on découpe le texte en **liste de paragraphes** et on étudie la structure de chaque paragraphe
- on découpe le texte en **liste de phrases** et on étudie la structure de chaque phrase
- enfin, on étudie le **vocabulaire**: sa *densité lexicale*, sa distribution dans les trois textes.

On commencera par des opérations basiques en Python, pour on finira par introduire quelques techniques de base en traitement automatisé du langage (TAL).

## Compétences

Notre analyse permet d'introduire au TAL et aux bases de l'utilisation de Python pour la fouille de texte:

- **les bases de la syntaxe** Python
- **manipulation des types de données basiques** en Python: chaînes de caractères, nombres entiers et décimaux, listes et dictionnaires
- **expressions régulières** (`regex`) pour détecter des motifs dans du texte
- **traitement automatisé du langage, avec `nltk`** (enfin, une très très très rapide introduction à quelques concepts de TAL)
- **utilisation de librairies** Python
- **calculs statistiques basiques**

---

# Poser les bases

On commence par installer les **librairies** nécessaires. En Python, certaines fonctionnalités sont disponibles de base, mais la plupart sont organisées dans des libraires. Une librairie est un ensemble de fonctions rassemblées ensemble avec un but spécifique: visualisations, calcul... Certaines librairies sont disponibles de base en Python, d'autres sont développées par des tiers et doivent être installées. Une librairie peut aussi être appelée "module" ou "package".

Les fichiers sont installés avec la commande `pip install <package name>` dans un terminal, ou `!pip install` dans un notebook.

In [ ]:
!pip install click==8.1.7
!pip install contourpy==1.1.1
!pip install cycler==0.12.1
!pip install fonttools==4.49.0
!pip install importlib-resources==6.1.2
!pip install joblib==1.3.2
!pip install kiwisolver==1.4.5
!pip install matplotlib==3.7.5
!pip install nltk==3.8.1
!pip install numpy==1.24.4
!pip install packaging==23.2
!pip install pillow==10.2.0
!pip install pyparsing==3.1.1
!pip install python-dateutil==2.9.0.post0
!pip install pytz==2024.1
!pip install regex==2023.12.25
!pip install six==1.16.0
!pip install tabulate==0.9.0
!pip install tqdm==4.66.2
!pip install tzdata==2024.1
!pip install Unidecode==1.3.8
!pip install zipp==3.17.0

Ensuite, on importe les libraires nécessaires pour les utiliser dans notre chaîne de traitement.

In [ ]:
from nltk.corpus import stopwords
from unidecode import unidecode
from tabulate import tabulate
import statistics
import random
import nltk
import os
import re

Pour importer une librairie, on utilise la commande `import`:

```python
import <nom du paquet>
```

Les librairies sont souvent organisées en *sous-modules* (comme les chapitres d'un livre). Il est donc aussi possible d'importer seulement un *module*, ou une seule fonction:

```python
from <nom_du_paquet> import <nom_du_module>
from <nom_du_paquet>.<nom_du_module> import <nom_du_module_ou_fonction>
```

Ensuite, `nltk` (la librairie faite pour le traitement du langage naturel) demande de faire quelques téléchargements:



In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger')

---

# Lire les fichiers

(Attention, il faut que les [trois fichiers](https://github.com/paulhectork/cours_ens2024_fouille_de_texte/tree/main/in) aient été téléchargés au format `.txt` dans la partie `Fichiers` de Google Collab, sans les renommer !)

On commence donc par lire le contenu de nos textes.

In [ ]:
def lire(nom_de_fichier):
    with open(nom_de_fichier, mode="r" ) as fh:
        corpus = fh.read()
    return corpus

Qu'est-ce qu'il se passe au dessus??

Pour **lire un fichier avec Python**, 2 étapes sont nécessaires: on ouvre le fichier, puis on lit son contenu et on l'assigne à une variable.

- pour ouvrir, on utilise: `open(<nom_du_fichier>, mode="r")`
- `with open(...) as fh` permet d'ouvrir un fichier et de l'assigner à la variable `fh`. Le fichier est automatiquement fermé à la fin du bloc.
- `fh.read()` permet de lire le contenu du fichier.

Pour éviter d'avoir à réécrire du code, **on utilise des fonctions**.
- Comme en maths, une fonction prend un ou plusieurs arguments, effectue des opérations et retourne une ou plusieurs valeurs.
- `def` marque le début de la définition d'une fonction, `return` la fin.
- `return` est optionnel. Ce mot-clé désigne le **résultat d'une fonction**. À la fin de l'exécusion d'une fonction, toutes les valeurs créés ou modifiées pendant l'exécution sont supprimées. Seules les valeurs retournées avec `return` peuvent être accédées en dehors de la fonction et utilisée ailleurs.
- Une fonction peut être appelée plus tard (ça évite d'avoir à réécrire du code, la flemme étant un moteur majeur de lae développeur.euse)
- Elle permet aussi de lancer la même opération avec différents arguments en entrée.

```python
def sommecarre(x,y): # sommecarre est le nom de la fonction, `x` le premier argument et `y` le 2e
    a = x*x          # on fait une opération
    b = y*y          # une deuxième opération
    return  a+b      # on calcule la somme et on la retourne. `a` et `b` ne sont pas accessibles en dehors de la fonction
z = sommecarre(3,7)  # on appelle la fonction. x=3 et y=7
```

In [ ]:
# qu'est-ce qui se passe ici?
dalloway = lire("./mrs_dalloway.txt")
lighthouse = lire("./to_the_lighthouse.txt")
waves = lire("./the_waves.txt")

print(waves)

# Préparer le texte

Pour faciliter l'analyse, on a commencer par simplifier un peu le texte.

En programmation, les données sont classées en différents "types", et chaque type de données permet certaines opérations, et vient avec des fonctions qui lui sont propres. On peut par exemple diviser un nombre entier, mais on ne peut pas diviser du texte. La fonction `type()` permet de connaître le type de donnée d'une variable.

Notre texte est une chaîne de caractères (`string`) et des fonctions sont prédéfinies pour le manipuler. On les utilise ici:

In [ ]:
def simplify(txt):
    """
    simplifier les 3 romans
    """
    txt = txt.lower() # supprimer les majuscules

    # supprimer les points qui ne séparent pas 2 phrases
    txt = txt.replace("mrs.", "mrs")  # syntaxe: `replace(<texte à remplace>, <remplacement>)`
    txt = txt.replace("ms.", "ms")
    txt = txt.replace("mr.", "mr")
    txt = txt.replace("dr.", "dr")
    txt = txt.replace("'s", "")
    # on peut l'écrire d'autres manières:
    # txt = txt.replace("mrs.", "mrs").replace("ms.", "ms").replace("mr.", "mr").replace("dr.", "dr").replace("'s", "")

    # supprimer les accents des lettres accentuées
    txt = unidecode(txt)  # unidecode n'existe pas par défaut, cette fonction a été installée avec la librairie `unidecode`

    return txt

waves = simplify(waves)
dalloway = simplify(dalloway)
lighthouse = simplify(lighthouse)

print(lighthouse)


---

# Diviser le texte en plus petites unités

Nos textes sont des `string`. Un ordinateur lit une `string` de façon bête et méchante: caractère après caractère. Pour l'instant, on ne peut donc faire d'analyses qu'au niveau de l'ensemble du texte, alors qu'on voudrait travailler à des plus petites échelles: le paragraphe et la phrase. Il va donc falloir diviser ce texte en plus petites unités, à l'aide de **listes**.

**Une liste est une série ordonnée de valeurs**.
- Les items d'une liste sont séparés par des virgules (`,`)
- Une liste et s'écrit entre `[]`. - Une liste peut contenir tout type de données: `string`, nombres entiers (`int`) et décimaux (`float`), mais aussi d'autres listes et des dictionnaires (`dict`, que l'on verra plus bas).
- On accède aux items d'une liste sont par leur index, c'est-à-dire leur position dans la liste. L'indexation commence à 0: le premier item est à la position 0, le second à 1... C'est étrange au début mais on s'y fait.

In [ ]:
sandwich = [ "pain", ["tofu", "tomate séchée", "roquette"], "pain" ]  # cette liste contient, dans l'ordre, une `string`, une `list`, une `string`
print(sandwich[0])     # on accède au 1er élément
print(sandwich[-1])    # si on fait `-n`, on accède au `nième` élement en partant de la fin de la liste. `-1` = le dernier élément de la liste !
print(sandwich[1][1])  # le 2e élément du 2e élément de notre liste: "tomate séchée"

On va donc écrire des fonctions pour scinder nos textes en listes de paragraphes et en listes de phrases. On commence par des petits bouts de code avant de tout rassembler dans des fonctions. Pour scinder un texte en liste, on utilise la méthode `.split("<caractère>")`.

In [ ]:
print(waves.split(" ")[:10])  # une liste des 10 premiers mots de The Waves

Pour faire **une liste de paragraphes**, on `split` le texte en lignes vides (en informatique, `\n` est un saut de ligne, `\n\n` une ligne vide).

In [ ]:
print(dalloway.split("\n\n")[3])         # qu'est-ce qu'on affiche ici?
print(lighthouse.split("\n\n")[10:-10])  # et ici?

Pour faire **une liste de phrases**, c'est un peu plus compliqué. Comme on a simplifié le texte, un `.` représente toujours une fin de phrase. Mais `?` et `!` sont aussi des fins de phrases. Il y a donc plusieurs séparateurs à détecter.

On va donc utiliser une **expression régulière** (regex pour les intimes). Les regex sont très répandues en informatique, et offrent une syntaxe très puissante (mais indigeste) pour détecter des motifs dans du texte.
- L'idée est de ne pas s'intéresser au contenu d'un texte, mais à sa structure: par exemple, 02/05/1972 et 10/12/1999 correspondent au motif: `[2 chiffres]/[2chiffres]/[4 chiffres]`. Il est alors possible de cibler ces deux dates par le même motif.
- Les regex définissent (entre autres):
  - **des quantificateurs**: ils permettent de quantifier le nombre de fois que l'on cherche un caractère:
    - `x?`: 0 ou 1 fois `x`,
    - `x+`: 1 ou plusieurs fois `x`,
    - `x*`: de 0 à n fois x...
  - **des classes de caractères**:
    - `\d`, par exemple, représente "n'importe quel chiffre";
    - `\s` est un espace
    - `.` est n'importe quel caractère;
    - `^` est le début d'une chaîne de caractères
    - `$` est la fin d'une chaîne de caractère.
  - **des opérateurs d'aleternance** ("caractère A ou B" correspond à: `[xy]` ou `(x|y)`)
- *pour une présentation plus détaillée, [voir ici](https://github.com/paulhectork/cours_ens2023_fouille_de_texte/blob/main/1_fouille_texte.ipynb)*

On veut scinder notre texte par `.`, `?` ou `!`. On utilise donc l'opérateur d'alternance `[]`, et on obtient la regex: `[\.?!]` (dans une regex, le point doit être précédé de `\`, sinon `.` est interprété comme *n'importe quel caractère*).

Pour *splitter* avec une regex, on utilise la fonction `split()` de la librairie `re` (librairie consacrée aux regex):

In [ ]:
print(re.split("[\.?!]", waves)[:10])       # qu'est-ce que j'affiche?
print(len( re.split("[\.?!]", dalloway) ))  # len() permet donne la longueur d'une liste ! comment interpréter ce résultat ?

On combine tout ce qu'on a vu en une fonction, `splitter()`, qui prend en argument `txt` un texte (chaîne de caractère) et retourne deux listes: une liste de paragraphes et une liste de phrases.

In [ ]:
def splitter(txt):
    """
    diviser le texte en unités distinctes: paragraphes et phrases.
    prend `txt`, une chaîne de caractères en entrées
    retourne `txt_paragraphe`, une liste de tous les paragraphes de ce texte
             et `txt_phrase`, une liste de toutes les phrases de ce texte
    """
    txt_paragraphe = []
    splitted = txt.split("\n\n")       # `.split()` produit ici une liste de paragraphes)
    for t in splitted:                 # itérer sur chaque paragraphe. `t` = string contenant un paragraphe
        if not re.search("^\s*$", t):  # ne pas prendre en compte les paragraphes vides
            txt_paragraphe.append(t)   # ajouter l'item à la liste

    txt_phrase = []
    txt = txt.replace("\n", " ")       # on supprime les sauts de ligne
    for t in re.split("[\.?!]", txt):  # re.split() permet de séparer une chaîne de caractères en listes en utilisant une regex. ici, `[\.?!]`, c'est à dire "." ou "?" ou "!"
        if not re.search("^\s*$", t):
            txt_phrase.append(t)

    return txt_paragraphe, txt_phrase

waves_paragraphe, waves_phrase = splitter(waves)
dalloway_paragraphe, dalloway_phrase = splitter(dalloway)
lighthouse_paragraphe, lighthouse_phrase = splitter(lighthouse)

print(waves_paragraphe[10])
print(lighthouse_phrase[-3:])

---

# Étudier les paragraphes

On a donc un texte nettoyé (`waves`, `dalloway`, `lighthouse`) et scindé en paragraphes (`*_paragraphe`) et en phrases `*_phrase`). On va donc pouvoir commencer notre analyse !

Tous les résultats seront stockés dans le type de données **dictionnaire (`dict`)**.
- Comme dans un vrai dictionnaire où un mot est associé à sa définition, un dictionnaire est un type de donnée permettant d'associer **une clé à une valeur**.
- Sa **syntaxe** est: `{ "clé1": <valeur 1>, "clé2": <valeur2> }`.
- Comme une liste, un dictionnaire peut contenir en valeurs n'importe quel type de données: `string`, `list`, autres `dict`... On peut donc représenter des objets très complexes en dictionnaires.
- On accède aux valeurs à partir des clés: `dico["clé1"]`.

In [ ]:
sandwich = {
    "pain": { "type":"pain pita", "quantité": 2 },
    "ingrédients": [ "houmous", "falafel", "aubergines"
                   , "zestes de citron", "coriandre" ]
}
print(sandwich["pain"])              # la valeur associée à `pain`
print(sandwich["pain"]["quantité"])  # sandwich -> pain -> quantité
print(sandwich["ingrédients"][3])    # et là, qu'est-ce qui s'affiche?

On créé donc 3 dictionnaires vides pour stocker les résultats de notre analyse des trois romans.

In [ ]:
waves_stats = {}
dalloway_stats = {}
lighthouse_stats = {}

On analyse la structure de nos paragraphes:
- quel est le nombre médian de mots par paragraphes?
- quel est le nombre médian de phrases par paragraphes?

Le nombre médian de mots par paragraphes, c'est facile:

In [ ]:
count_mots = []                 # liste contenant le nombre de mots pour chaque paragraphe du texte
for p in dalloway_paragraphe:   # on accède à chaque paragraphe de mrs. dalloway
  p = p.split(" ")              # on fait du paragraphe une liste de mots
  nb_mots = len(p)              # la longueur de la liste de mots = le nombre de mots du paragraphe
  count_mots.append(nb_mots)
print(statistics.median(count_mots))  # statistics.median() permet de calculer la valeur médiane d'une liste de nombres

Pour calculer le nombre médian de phrases par paragraphes, une petite subtilité: on devra supprimer les paragraphes vides (c'est-à-dire, qui ne contiennent que des espaces ou rien):

In [ ]:
count_phrases = []                      # liste du nombre de phrases par paragraphe
for p in dalloway_paragraphe:
  nb_phrases = 0
  p = re.split("[\.?!]", p)             # on fait une liste de paragraphes
  for phrase in p:
    if not re.search("^\s*$", phrase):  # que veut dire cette ligne?
      nb_phrases += 1                   # on incrémente le compteur
  count_phrases.append(nb_phrases)
print(statistics.median(count_phrases))


On a calculé le nombre médian de mots par paragraphes et le nombre médian de phrases par paragraphe: il ne reste plus qu'à faire une fonction pour faire les calculs sur les 3 romans !

In [ ]:
def study_paragraphe(paragraphes, stats):
    """
    analyser la structure d'un paragraphe:
    longueur médiane d'un paragraphe en nombres de mots et en nombre de phrases

    :param paragraphes: une liste contenant tous les paragraphes d'un texte
    :param stats      : le dictionnaire contenant les statistiques sur un texte
    """
    count_mots = []     # liste avec le nombre de mots pour chaque paragraphe
    count_phrases = []  # liste du nombre de mots phrases par paragraphe

    # on calcule le nombre médian de mots par paragraphe
    for p in paragraphes:
        p = p.split(" ")           # on transforme le paragraphe en une liste de mots
        count_mots.append(len(p))  # on ajoute à `counts_mots` `len(p)`, soit le nombre d'items dans la liste `p`
    med_mots = statistics.median(count_mots)

    # nombre médian de phrases par paragraphes
    for p in paragraphes:
        nb_phrases = 0
        p = re.split("[\.?!]", p)               # on fait une liste de paragraphes
        for phrase in p:
            if not re.search("^\s*$", phrase):  # on ne prend pas en compte les lignes vides
                nb_phrases += 1                 # on incrémente le compteur
        count_phrases.append(nb_phrases)
    med_phrases = statistics.median(count_phrases)

    print(med_mots, med_phrases)
    stats["nombre médian de mots par paragraphes"] = med_mots
    stats["nombre médian de phrases par paragraphes"] = med_phrases
    return stats

stats_waves = study_paragraphe(waves_paragraphe, waves_stats)
stats_dalloway = study_paragraphe(dalloway_paragraphe, dalloway_stats)
stats_lighthouse = study_paragraphe(lighthouse_paragraphe, lighthouse_stats)
print("the waves: nombre médian de mots par paragraphes    : ", waves_stats["nombre médian de mots par paragraphes"])
print("the waves: nombre médian de phrases par paragraphes : ", waves_stats["nombre médian de phrases par paragraphes"])


---

# Étudier les phrases

On a fini la première échelle de notre étude: l'étude paragraphe par paragraphe :)) On va donc pouvoir commencer à étudier la structure des phrases dans les trois romans. On va s'intéresser à deux indicateurs:

- le nombre médian de mots par phrases (une variante de ce qu'on a fait plus haut)
- le nombre moyen de signes de ponctuations par phrases. Cet indicateur permettra de regarder comment évolue la complexité des phrases, mais aussi à approximer le nombre de propositions dans une phrase. On pourra donc voir l'évolution de la complexité d'une phrase.
  - Ici, on préfère la moyenne à la médiane parce que la médiane est déséquilibrée par le grand nombre de phrases sans signes de ponctuation. La moyenne permet mieux de représenter ce qui se passe au niveau des phrases plus complexes, avec plusieurs signes de ponctuation.

On commence par le nombre médian de mots par phrases:

In [ ]:
count_mots = []
for p in dalloway_phrase:
    counter = 0
    p = p.split(" ")                      # on fait de `p` une liste de mots
    for mot in p:
        if not re.search("^\s*$", mot):   # on ne compte pas les éléments vides de la liste de mots
            counter += 1
    count_mots.append(counter)
med_mots = statistics.median(count_mots)
print(med_mots)  # nombre médian de mots par phrases pour Mrs. Dalloway


Ensuite, on calcule le nombre moyen de signes de ponctuation par phrase, pour un roman. La logique est similaire à ce qu'on a fait avant:

- on intère sur chaque phrase d'un roman
- on extrait les chiffres pertinents pour cette phrase
- on l'ajoute à `count_punct`, une liste contenant toutes les données pour chaque phrase
- on calcule une moyenne de toutes les valeurs de `count_punct`.

Pour reprérer les signes de ponctuation, on utilise la regex suivante:

```
(-{2,}|[,;:&—\(])
```
- `(<a>|<b>)`: soit `<a>`, soit `<b>`
  - `<a>` = `-{2,}`: deux tirets ou plus (pour représenter un cadratin)
  - `<b>` = `[,;:&—\(]`: un de ces signes de ponctuation. (`(` est précédé d'un `\` car c'est un caractère spécial dans une regex).
- on cherche donc soit deux tirets, soit un des signes de ponctuation entre deux crochets.
- on utilise `re.findall()`, qui permet d'obtenir une liste de toutes les occurrences trouvées dans une `string`.




In [ ]:
count_punct = []
for p in waves_phrase:
    punct = re.findall("([,;:&—\(]|-{2,})", p)
    count_punct.append(len(punct))      # on ajoute le nb de signes de ponctuations pour chaque phrase
moyenne = statistics.mean(count_punct)
print( round(moyenne, 3) )  # `round()` donne l'arrondi d'un nombre décimal. le 1e argument est le nombre à arrondir, le 2e argument est le nombre de chiffres après la virgules


Nos opérations de calcul sont prêtes ! On peut donc les combiner dans une chouette fonction:


In [ ]:
def study_phrase(phrases, stats):
    """
    analyser la structure d'une phrase: longueur médiane
    d'une phrase, nombre de signes de ponctuation
    """
    # nombre médian de mots par phrases
    count_mots = []
    for p in phrases:
        p = p.split(" ")                                    # on fait de `p` une liste de mots
        p = [ x for x in p if not re.search("^\s*$", x) ]   # on enlève les éléments vides de la liste
        count_mots.append(len(p))                           # on compte le nombre de mots dans la liste et on les ajoute à notre compteur
    med_mots = statistics.median(count_mots)

    # moyenne de signes de ponctuation par phrases.
    count_punct = []
    for p in phrases:
        punct = re.findall("([,;:&—\(]|-{2,})", p)          # re.findall() retourne une liste de toutes les occurences de la regex trouvées
        count_punct.append(len(punct))
    mean_punct = round(statistics.mean(count_punct), 3)

    stats["nombre médian de mots par phrase"] = med_mots
    stats["nombre moyen de signes de ponctuation par phrase"] = mean_punct
    return stats

stats_waves = study_phrase(waves_phrase, waves_stats)
stats_dalloway = study_phrase(dalloway_phrase, dalloway_stats)
stats_lighthouse = study_phrase(lighthouse_phrase, lighthouse_stats)

print("to the lighthouse: nombre médian de mots par phrase                 : ", stats_lighthouse["nombre médian de mots par phrase"])
print("to the lighthouse: nombre moyen de signes de ponctuation par phrase : ", stats_lighthouse["nombre moyen de signes de ponctuation par phrase"])


# Étudier le vocabulaire: la *densité lexicale*

## Définitions

Jusqu'à maintenant, on est resté sur des méthodes d'analyse assez basiques, centrées sur la détection de motifs dans le texte. Ça nous a permis d'introduire quelques bases de Python, mais on est assez loin de produire quelque chose de vraiment intéressant.

À partir de maintenant, on va utiliser des méthodes de TAL, avec la librairie Python dédiée, `nltk`. Il existe tout plein de méthodes de traitement du langage (reconnaissance d'entités nommées, *topic modelling*, *sentiment analysis*...)

Ici, on va s'intéresser à la [densité lexicale](https://en.wikipedia.org/wiki/Lexical_density) de chaque texte. La densité lexicale vise à mesurer la complexité informationnelle d'un texte, écrit ou oral. Elle s'appuie sur la classification des mots en deux catégories:

- les **mots-lexicaux** (`lexical units`) regroupent l'ensemble de mots "porteurs d'information" dans un texte: les noms, les verbes, les adverbes et les adjectifs qualitatifs. On considère que c'est grâce à eux que l'information est transmise.
- les **mots-outils ou mots-grammaticaux** (`function words`) regroupent tout le reste des mots. On considère qu'ils sont moins porteurs de sens et qu'ils servent surtout à la structure de la phrase.

La classification entre ces deux groupes n'est pas clairement définie et peut varier d'une étude à l'autre.

La **densité lexicale** (notée $L_d$) mesure, sur une échelle de 0 à 100, la proportion de mots lexicaux dans un texte. Plus $L_d$ est proche de 100, plus il y a de mots lexicaux, plus on considère que le texte est riche en information. On considère en moyenne que que:
- pour un texte écrit, $L_d > 40$, et pour un texte oral, $L_d < 40$.
- pour un texte de fiction, $ 40 < L_d < 54$; pour un texte de non-fiction, $40 < L_d < 65$.

Elle se mesure ainsi (avec $N_{lex}$ le nombre de mots lexicaux et $N$ le nombre total de mots dans un texte:

$L_d = (\frac{N_{lex}}{N}) \times 100$

PS: il existe plusieurs variantes de calcul, mais on utilise celle définie par Ure J. (1971).

## Chaîne de traitement

Avec `nltk`, calculer une densité lexicale est vraiment très très simple:

- on tokenise notre texte en `tokens` (items lexicaux)
- on fait un [`POS tag`](https://fr.wikipedia.org/wiki/%C3%89tiquetage_morpho-syntaxique) (*part-of-speech tagging*) pour déterminer la fonction de chaque *token* dans le texte: pronom, verbe...
- on compte le nombre d'occurrences pour chaque fonction des tokens d'un texte
- à partir de là, on calcule notre $L_d$

Les tags possibles pour notre étiquetage morpho-syntaxique sont:

```
Tag  | Meaning             | English Examples
~~~~~|~~~~~~~~~~~~~~~~~~~~~|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ADJ  | adjective           | new, good, high, special, big, local
ADP  | adposition          | on, of, at, with, by, into, under
ADV  | adverb              | really, already, still, early, now
CONJ | conjunction         | and, or, but, if, while, although
DET  | determiner, article | the, a, some, most, every, no, which
NOUN | noun                | year, home, costs, time, Africa
NUM  | numeral             | twenty-four, fourth, 1991, 14:24
PRT  | particle            | at, on, out, over per, that, up, with
PRON | pronoun             | he, their, her, its, my, I, us
VERB | verb                | is, say, told, given, playing, would
.    | punctuation marks   | . , ; !
X    | other               | ersatz, esprit, dunno, gr8, univeristy
```



In [ ]:
tokens = nltk.word_tokenize(waves)              # tokenisation au mot (similaire à txt.split(" "), mais performe des simplifications en plus)
print(tokens)

size = len(tokens)                              # on travaille sur tout le corpus
pos = nltk.pos_tag(tokens, tagset="universal")  # part-of-speech tagging. universal définit des classes très généralistes
print(pos[:10])                                 # que voit-on ici?
tags = []
for (token, tag) in pos:
    tags.append(tag)

fd = nltk.FreqDist(tags)  # `nltk.FreqDist()` associe à chaque valeur distincte d'une liste le nombre d'occurrences dans cette liste
fd.tabulate()

nlex = fd.get("NOUN") + fd.get("VERB") + fd.get("ADJ") + fd.get("ADV")  # nb d'unités lexicales
ld = 100 * (nlex/size)

On écrit une fonction qui reprenne tout ça et on la lance:

In [ ]:
def densite_lexicale(txt, stats):
    """
    enfin, on étudie la densité lexicale de chaque roman

    on suit la méthode de Ure: 100 * <nb d'unités lexicales> / <nb de tokens>
    https://en.wikipedia.org/wiki/Lexical_density
    https://www.nltk.org/book/ch05.html
    """
    tokens = nltk.word_tokenize(txt)    # tokenisation au mot (similaire à txt.split(" "), mais performe des simplifications en plus)

    size = len(tokens)  # on travaille sur tout le corpus
    pos = nltk.pos_tag(tokens, tagset="universal")  # part-of-speech tagging (classification du texte en classes: verbes...). universal définit des classes très généralistes
    tags = []
    for (token, tag) in pos:
        tags.append(tag)

    fd = nltk.FreqDist(tags)  # valeur associée aux nombre d'occurrences de celle-ci
    nlex = fd.get("NOUN") + fd.get("VERB") + fd.get("ADJ") + fd.get("ADV")  # nb d'unités lexicales
    ld = 100 * (nlex/size)
    print(ld)

    stats["densité lexicale"] = round(ld, 3)

    return stats

stats_lighthouse = densite_lexicale(lighthouse, lighthouse_stats)
stats_dalloway = densite_lexicale(dalloway, dalloway_stats)
stats_waves = densite_lexicale(waves, stats_waves)


---

# Afficher les résultats

La base de notre analyse est finie ! Il ne reste plus qu'à afficher les résultats:)

Pour ce faire, on utilise la fonction `tabulate.tabulate()`, qui prend deux arguments: `headers` (une liste d'en-têtes) et `data` (une liste de listes, avec une liste interme par ligne).

In [ ]:
headers = [ "", "Mrs. Dalloway, 1925", "To the Lighthouse, 1927", "The Waves, 1931" ]
data = [ [ "nombre médian de mots par paragraphes"
         , stats_dalloway["nombre médian de mots par paragraphes"]
         , stats_lighthouse["nombre médian de mots par paragraphes"]
         , stats_waves["nombre médian de mots par paragraphes"]
         ],
         [ "nombre médian de phrases par paragraphes"
         , stats_dalloway["nombre médian de phrases par paragraphes"]
         , stats_lighthouse["nombre médian de phrases par paragraphes"]
         , stats_waves["nombre médian de phrases par paragraphes"]
         ],
         [ "nombre médian de mots par phrase"
         , stats_dalloway["nombre médian de mots par phrase"]
         , stats_lighthouse["nombre médian de mots par phrase"]
         , stats_waves["nombre médian de mots par phrase"]
         ],
         [ "nombre moyen de signes de ponctuation par phrase"
         , stats_dalloway["nombre moyen de signes de ponctuation par phrase"]
         , stats_lighthouse["nombre moyen de signes de ponctuation par phrase"]
         , stats_waves["nombre moyen de signes de ponctuation par phrase"]
         ],
         [ "densité lexicale"
         , stats_dalloway["densité lexicale"]
         , stats_lighthouse["densité lexicale"]
         , stats_waves["densité lexicale"]
         ]
]
print(tabulate(data, headers, tablefmt="rounded_grid"))

Qu'est-ce qu'on peut dire de ces résultats?

---

# (Bonus?) Étudier le vocabulaire: distribution du vocabulaire

Pour finir, on va étudier la **distribution du vocabulaire** dans les texte. **On classe les tokens en déciles**, des mots les plus utilisés aux mots les moins utilisés. Le but, c'est de voir si l'autrice utilise beaucoup de mots "rares", ou si au contraire son vocabulaire se constitue au contraire d'une classe de mots qu'elle répère très souvent.

On va calculer deux mesures:

- la moyenne d'occurrences d'un item dans un texte, par décile
- le nombre d'items distincts par un décile

Le processus est le suivant:

1. supprimer les **caractères non-alphabétiques**
2. tokeniser le texte
3. supprimer les **"stopwords"** (ensemble de mots souvent présents qui déséquilibrent l'étude du vocabulaire)
4. faire un **`pos tagging`** et créer un **`sample` de 5000 mots-contenu** pour chaque texte
5. **lémmatiser** notre sample. La [lémmatisation](https://fr.wikipedia.org/wiki/Lemmatisation), c'est extraire le *lemme*, c'est-à-dire la forme canonique, d'un mot. Par exemple: `était` -> `être`
6. calculer une **distrubtion de fréquence** de nos lemmes
7. **classer notre corpus en déciles**. Pour chaque décile, calculer le nombre de lemmes qu'il contient et la moyenne du nombre de fois où chaque lemme est utilisé.

C'est un processus plus long, mais qui mélange tout ce qu'on a vu plus haut, qui mélange l'utilisation de `nltk` et de structures de données basiques en Python.

### Étape 1 et 2
D'abord, on **procède au nettoyage** et on **tokenise le texte**:

In [ ]:
txt = lighthouse
print(txt[:100])

# 1)
# `[^<motif>]` veut dire "tout sauf <motif>".
# `[^a-z ]` veut donc dire "tout ce qui n'est pas un espace ou un caractère alphabétique"
txt  = re.sub("[^a-z ]", " ", txt)  # que fait donc cette ligne?
txt = re.sub("\s+", " ", txt)       # on normalise les espaces
print(txt[:100])

# 2) on tokenise
tokens = nltk.word_tokenize(txt)
print(tokens[:100])
print(len(tokens))

### Étape 3
Ensuite, **on supprime les stopwords**


In [ ]:
# 3) on supprime les stopwords
stop_words = list(stopwords.words("english"))
print(stop_words[:10])

tokens_filtered = []
for token in tokens:
    if token.lower() not in stop_words:
        tokens_filtered.append(token)
print(tokens_filtered[:100])
print(len(tokens_filtered))

# en une ligne, comment calculer le nombre de tokens qui ont été supprimées?

### Étape 4
Pour continuer, on fait le **`pos tagging`et on crée notre corpus de 5000 mots-contenu**.

In [ ]:
size = 5000  # la taille du corpus final: 5000 tokens
sample_pos = []
pos = nltk.pos_tag(tokens_filtered, tagset="universal")  # part-of-speech tagging (classification du texte en classes: verbes...). universal définit des classes très généralistes
for (token, tag) in pos:
    if tag in [ "NOUN", "VERB", "ADJ", "ADV" ]:  # si ce token est un mot-contenu
        sample_pos.append(token)
sample_pos = random.sample(sample_pos, size)  # on créé une liste de 5000 items créés au hasard dans `sample_pos`

print(sample_pos[:100])
print(len(sample_pos))

### Étape 5

**On lemmatise**. Comme pour le `pos tagging`, `nltk` propose des outils tout prêts, et plusieurs lemmatiseurs. On utilise le `WordNetLemmatizer`, entraîné sur le jeu de données [`WordNet`](https://wordnet.princeton.edu/)**texte en gras**

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
sample_lem = [ lemmatizer.lemmatize(token) for token in sample_pos ]  # exemple d'itération en une ligne: on boucle sur tous les items de `sample_pos` et on leur applique une fonction
print(sample_lem)
print(len(set(sample_lem)))  # le nombre de lemmes uniques dans notre liste de 5000 lemmes (`set` est un type de données équivalent à une ligne, mais sans doublons. utiliser `len()` sur une liste permet donc de supprimer les doublons de la `list`)

### Étape 6

On calcule notre distribution de fréquences. C'est tout simple, comme on l'a déjà vu:

In [ ]:
fd = nltk.FreqDist(sample_lem)
fd.tabulate()  # les 10 lemmes les plus utilisés

fd.plot(title="To the lighthouse")  # ça prend un peu de temps et c'est pas nécessairement ultra lisible

## Étape 7

Comme on l'a vu avec le graphique ci-dessus, pour comparer les distributions de fréquences entre nos 3 romans, pour le moment, c'est pas donné: la courbe contient plusieurs milliers de valeurs, ce qui rend l'analyse à l'oeil nu difficile.

On va donc **classer les lemmes en 10 groupes**, des 10% des lemmes les plus utilisés au 10% des lemmes les moins utilisés. C'est la partie la plus compliquée du code, mais elle ne comporte que des choses que l'on a déjà vues !

In [ ]:
distribution_moyenne = []  # [ <moyenne d'occurrences pour un lemme, par décile> ]
distribution_somme = []    # [ <nombre absolu de lemmes, par décile> ]

for i in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:  # on itère sur chaque décile
    k = []  # liste du nombre d'occurrences pour chaque lemme, pour le décile actuel

    # on remplit `k` avec le nombre d'occurrences par quartile
    for mot, occurrences in fd.items():          # `fd.items()` permet de boucler sur des couples `<lemme>`: `<nombre d'occurrences>`
        print(mot, occurrences)

        quantile = occurrences/max(fd.values())  # représentation de l'utilisation du mot sur une échelle 0..1: 0 = mot jamais utilisé, 1 = mot le plus utilisé

        # on vérifie si `quantile` correspond au quantile actuel `i`.
        # si oui, on ajoute à `k`, notre liste du nombre d'occurrences,
        # le nombre d'occurrences de ce lemme
        if i != 1:
            if i > quantile >= i-0.1:
                k.append(occurrences)
        else:
            if i >= quantile >= i-0.1:
                k.append(occurrences)

    # on a fini de remplir `k` pour ce décile avec tous les lemmes du roman
    # on calcule nos statistiques et on les ajoute aux listes `distribution`
    if len(k) > 0:
        mean = round(statistics.mean(k), 3)
        distribution_moyenne.append(mean)
    else:
        distribution_moyenne.append(0)  # 0 mot ne rentre dans ce quantile => on ne peut calculer de moyenne
    distribution_somme.append(len(k))

print(distribution_somme)
print(distribution_moyenne)
print(distribution_moyenne[-1])  # comment interpréter ce nombre?
print(distribution_somme[2])      # et celui-ci?

Et voilà ! Maintenant, on fait de tout ça une fonction, et ajoute les résultats à nos dictionnaires de statistiques.

In [ ]:
def distribution_vocabulaire(txt, stats, title):
    """
    étudier la distribution du vocabulaire dans les trois romans
    """
    # étape 1
    txt = re.sub("[^a-z ]", " ", txt)  # on enlève tous les caractères non-alphabétiques et les espaces
    txt = re.sub("\s+", " ", txt)      # on normalise les espaces

    # étape 2
    tokens = nltk.word_tokenize(txt)   # tokenisation au mot (similaire à txt.split(" "), mais performe des simplifications en plus)

    # étape 3
    # on supprime tous les stopwords (mots jugés
    # "inutiles" pour l'analyse automatique)
    tokens_filtered = []
    stop_words = set(stopwords.words("english"))
    for token in tokens:
        if token.lower() not in stop_words:
            tokens_filtered.append(token)

    # étape 4
    # on fait un part-of-speech tagging sur le
    # corpus pour pouvoir ensuite le lemmatiser
    size = 5000  # la taille du corpus final: 5000 tokens
    sample_pos = []
    pos = nltk.pos_tag(tokens_filtered, tagset="universal")  # part-of-speech tagging (classification du texte en classes: verbes...). universal définit des classes très généralistes
    for (token, tag) in pos:
        if tag in [ "NOUN", "VERB", "ADJ", "ADV" ]:
            sample_pos.append(token)
    sample_pos = random.sample(sample_pos, size)

    # étape 5
    # on lemmatise le corpus
    lemmatizer = nltk.stem.WordNetLemmatizer()
    sample_lem = [ lemmatizer.lemmatize(token) for token in sample_pos ]

    # étape 6
    # calculter une distribution de fréquences
    fd = nltk.FreqDist(sample_lem)  # mot / nombres d'occurrences
    fd.plot(title=title)

    # étape 7
    # grouper le vocabulaire en quantiles:
    # lecture: les 10% des lemmes les plus fréquemment
    # rencontrés sont utilisés en moyenne n fois.
    distribution_moyenne = []  # [ <moyenne d'occurrences pour un lemme, par décile>]
    distribution_somme = []    # [ <nombre absolu de lemmes, par décile>]
    for i in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        k = []  # liste du nombre d'occurrences par quantile
        # on remplit `k` avec le nombre d'occurrences par quartile
        for mot, occurrences in fd.items():
            quantile = occurrences/max(fd.values())  # représentation de l'utilisation du mot sur une échelle 0..1: 0 = mot jamais utilisé, 1 = mot le plus utilisé
            if i != 1:
                if i > quantile >= i-0.1:
                    k.append(occurrences)
            else:
                if i >= quantile >= i-0.1:
                    k.append(occurrences)
        # on calcule nos statistiques et on les ajoute aux listes `distribution`
        if len(k) > 0:
            mean = round(statistics.mean(k), 3)
            distribution_moyenne.append(mean)
        else:
            distribution_moyenne.append(0)  # 0 mot ne rentre dans ce quantile => on ne peut calculer de moyenne
        distribution_somme.append(len(k))

    # enfin, on crée des dictionnaires pour afficher les résultats de façon lisible
    deciles = [ "0..10", "10..20", "20..30", "30..40", "40..50"
              , "50..60", "60..70", "70..80", "80..90", "90..100" ]
    somme = {}
    moyenne = {}
    for i,d in enumerate(deciles):  # enumerate permet d'itérer sur une liste avec un couple un couple [index, valeur]
        somme[d] = distribution_somme[i]
        moyenne[d] = distribution_moyenne[i]
    stats["lemmes distincts"] = somme
    stats["lemme moyenne"] = moyenne
    return stats

stats_lighthouse = distribution_vocabulaire(lighthouse, lighthouse_stats, "To the lighthouse")
stats_dalloway = distribution_vocabulaire(dalloway, dalloway_stats, "Mrs. Dalloway")
stats_waves = distribution_vocabulaire(waves, stats_waves, "The Waves")


# Afficher les résultats définitifs

Ici encore, on utilise `tabulate`. Je rentre pas dans le détail de la fonction `.join()`. Elle permet de joindre les différentes valeurs d'un itérable par une chaîne de caractères, ici `\n`.

In [ ]:
headers = [ "", "Mrs. Dalloway, 1925", "To the Lighthouse, 1927", "The Waves, 1931" ]
data = [ [ "nombre médian de mots par paragraphes"
         , stats_dalloway["nombre médian de mots par paragraphes"]
         , stats_lighthouse["nombre médian de mots par paragraphes"]
         , stats_waves["nombre médian de mots par paragraphes"]
         ],
         [ "nombre médian de phrases par paragraphes"
         , stats_dalloway["nombre médian de phrases par paragraphes"]
         , stats_lighthouse["nombre médian de phrases par paragraphes"]
         , stats_waves["nombre médian de phrases par paragraphes"]
         ],
         [ "nombre médian de mots par phrase"
         , stats_dalloway["nombre médian de mots par phrase"]
         , stats_lighthouse["nombre médian de mots par phrase"]
         , stats_waves["nombre médian de mots par phrase"]
         ],
         [ "nombre moyen de signes de ponctuation par phrase"
         , stats_dalloway["nombre moyen de signes de ponctuation par phrase"]
         , stats_lighthouse["nombre moyen de signes de ponctuation par phrase"]
         , stats_waves["nombre moyen de signes de ponctuation par phrase"]
         ],
         [ "densité lexicale"
         , stats_dalloway["densité lexicale"]
         , stats_lighthouse["densité lexicale"]
         , stats_waves["densité lexicale"]
         ],
         [ "distribution du vocabulaire\n(nombre de lemmes distincts par décile)"
         , "\n".join(f"{k} : {v}" for k,v in stats_dalloway["lemmes distincts"].items() )
         , "\n".join(f"{k} : {v}" for k,v in stats_lighthouse["lemmes distincts"].items() )
         , "\n".join(f"{k} : {v}" for k,v in stats_waves["lemmes distincts"].items() )
         ],
         [ "distribution du vocabulaire\n(moyenne d'utilisation d'un lemme par décile)"
         , "\n".join(f"{k} : {v}" for k,v in stats_dalloway["lemme moyenne"].items() )
         , "\n".join(f"{k} : {v}" for k,v in stats_lighthouse["lemme moyenne"].items() )
         , "\n".join(f"{k} : {v}" for k,v in stats_waves["lemme moyenne"].items() )
         ]
]
print(tabulate(data, headers, tablefmt="rounded_grid"))
